# Text Generation With LSTM Recurrent Neural
http://www.gutenberg.org/cache/epub/11/pg11.txt

In [49]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.utils import to_categorical

In [50]:
import re

filename = 'book.txt'
file = open(filename, 'r', encoding='utf_8')
raw_text = file.read().lower()
file.close()
raw_text = re.sub(r'[.,"\'-?:!;]','', raw_text) 

In [51]:
chars = sorted(list(set(raw_text)))
char_to_int = {c:i for i, c in enumerate(chars)}

In [52]:
n_chars = len(raw_text)
n_vocab = len(chars)
n_chars, n_vocab

(158475, 40)

## Dataset preparation
split the book text up into subsequences with a fixed length of 100 characters, an arbitrary length. 

split the data by sentences, padding the shorter sequences and truncating the longer ones.

In [53]:
seq_len = 100
dataX = []
dataY = []
for i in range(n_chars-seq_len):
    seq_in = raw_text[i:i+seq_len]
    seq_out = raw_text[i+seq_len]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append([char_to_int[seq_out]])
n_patterns = len(dataX)
print('Number of patterns: ', n_patterns)

Number of patterns:  158375


In [54]:
X = np.reshape(dataX, (n_patterns, seq_len, 1))
X = X / n_vocab
y = to_categorical(dataY)

## Model

In [55]:
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.7),
    Dense(y.shape[1], activation='softmax')
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam())

In [56]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1238/1238 [==============================] - 352s 284ms/step - loss: 2.9839
Epoch 2/20
1238/1238 [==============================] - 341s 276ms/step - loss: 2.8308
Epoch 3/20
1238/1238 [==============================] - 325s 263ms/step - loss: 2.7602
Epoch 4/20
1238/1238 [==============================] - 324s 262ms/step - loss: 2.7116
Epoch 5/20
1238/1238 [==============================] - 324s 262ms/step - loss: 2.6740
Epoch 6/20
1238/1238 [==============================] - 321s 259ms/step - loss: 2.6410
Epoch 7/20
1238/1238 [==============================] - 329s 266ms/step - loss: 2.6084
Epoch 8/20
1238/1238 [==============================] - 325s 262ms/step - loss: 2.5759
Epoch 9/20
1238/1238 [==============================] - 323s 261ms/step - loss: 2.5423
Epoch 10/20
1238/1238 [==============================] - 328s 265ms/step - loss: 2.5093
Epoch 11/20
1238/1238 [==============================] - 326s 263ms/step - loss: 2.4759
Epoch 12/20
1238/1238 [==================

## Text generation

In [58]:
import sys

int_to_char = {i:c for i, c in enumerate(chars)}

start = np.random.randint(len(dataX)-1)
pattern = dataX[start]
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / n_vocab
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

 the wored to toen i shink to toen i shink to toen i shink to toen i think to toen i think to toen i think toen the roeee tf the soee a dor oo the soeee the gorse to toen i shink toee in the roeee th the would toen the roeee th the would toen the roeee th the would be an anl the world toene the worl  “ho a toie to toen to toen i shink to toen i shink to toen i shink to toen i think to toen i think toen the roeee tf the soee a dor oo the soeee the gorse to toen i shink toee in the roeee th the would toen the roeee th the would toen the roeee th the would be an anl the world toene the worl  “ho a toie to toen to toen i shink to toen i shink to toen i shink to toen i think to toen i think toen the roeee tf the soee a dor oo the soeee the gorse to toen i shink toee in the roeee th the would toen the roeee th the would toen the roeee th the would be an anl the world toene the worl  “ho a toie to toen to toen i shink to toen i shink to toen i shink to toen i think to toen i think toen the ro

## Larger LSTM

In [59]:
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dropout(0.2),
    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [60]:
model.fit(X, y, epochs=50, batch_size=64)

Epoch 1/50
2475/2475 [==============================] - 1460s 589ms/step - loss: 2.8397
Epoch 2/50
2475/2475 [==============================] - 1722s 696ms/step - loss: 2.4980
Epoch 3/50
2475/2475 [==============================] - 1685s 681ms/step - loss: 2.2565
Epoch 4/50
2475/2475 [==============================] - 1676s 677ms/step - loss: 2.0867
Epoch 5/50
2475/2475 [==============================] - 1732s 700ms/step - loss: 1.9711
Epoch 6/50
2475/2475 [==============================] - 1765s 713ms/step - loss: 1.8849
Epoch 7/50
2475/2475 [==============================] - 1708s 690ms/step - loss: 1.8175
Epoch 8/50
2475/2475 [==============================] - 1701s 687ms/step - loss: 1.7601
Epoch 9/50
2475/2475 [==============================] - 1672s 675ms/step - loss: 1.7157
Epoch 10/50
2475/2475 [==============================] - 1660s 671ms/step - loss: 1.6787
Epoch 11/50
2475/2475 [==============================] - 1661s 671ms/step - loss: 1.6413
Epoch 12/50
2475/2475 [=======

In [62]:
start = np.random.randint(len(dataX)-1)
pattern = dataX[start]
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / n_vocab
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

r that”

“i won’t think they don’t talk i con’t like the look of the sea and wasn’t the same thing” said the mock turtle “they all move that makes the more the same thing as ‘ou know about the sea the mock turtle i’m a real to go nn thle” said the mock turtle “they all move that makes the more the same thing as ‘ou know about the sea the mock turtle i’m a real to go nn thle” said the mock turtle “they all move that makes the more the same thing as ‘ou know about the sea the mock turtle i’m a real to go nn thle” said the mock turtle “they all move that makes the more the same thing as ‘ou know about the sea the mock turtle i’m a real to go nn thle” said the mock turtle “they all move that makes the more the same thing as ‘ou know about the sea the mock turtle i’m a real to go nn thle” said the mock turtle “they all move that makes the more the same thing as ‘ou know about the sea the mock turtle i’m a real to go nn thle” said the mock turtle “they all move that makes the more the same t